# Lending Club

**Описание проекта:**

Работа является тестовым заданием для зачисления на стажировку в [ШИФТ](https://team.cft.ru/events/130) (Школа информационных и финансовых технологий).

Исходные данные - это данные онлайн-заявителей [Lending Club](https://www.lendingclub.com/) в США, которые предоставляет финансовые услуги в сфере инвестирования и кредитования физических лиц. Данные уже разбиты на выборки:
- `X_test`  - контрольная/тестовая выборка,
- `X_train` - обучающая/тренировочная выборка,
- `y_train` - целевой признак для тренировочной выборки `loan_status`: **1** - кредит не возвращен, **0** - заемщик вернул кредит.

(*) целевой признак для тестовой выборки не предоставляется. Этот датасет используется для контроля выполнения задания и находится у организаторов конкурса.

Необходимо построить модель предсказания статус возврата кредита `loan_status`. Для каждого клиента из тестовой выборки `X_test.csv` составить предсказание и сохранить результат в виде датасета с колонками `index` и `loan_status`, где проставлена вероятность _не возврата кредита_. Название датасета: `answer.csv`, и должен быть упакован в zip-архив. Ниже приведёт пример содержания файла с ответами:

```
index,loan_status
1,0
2,0
3,0
5,0
7,0
9,0
10,0
17,0
```

Для оценка модели будет использоваться метрике `Gini`:
$$
Gini = 2 * RA - 1,
$$
где RA - это значение ROC AUC.

**План работы:**
1. Чтение и знакомство с данными
2. Предобработка данных
3. Анализ данных (EDA)
4. Модели предсказания
5. Вывод

## Чтение и знакомство с данными

In [1]:
# базовые библиотеки
import pandas as pd
import matplotlib.pyplot as plt

# спец. библиотеки
from sklearn.metrics import roc_auc_score

In [12]:
X_test  = pd.read_csv('X_test.csv',  index_col='index')
X_train = pd.read_csv('X_train.csv', index_col='index')
#y_train = pd.read_csv('y_train.csv')

In [13]:
X_train.head(10)

,acc_now_delinq,addr_state,annual_inc,chargeoff_within_12_mths,collections_12_mths_ex_med,delinq_2yrs,dti,earliest_cr_line,emp_length,fico_range_high,...,pub_rec,pub_rec_bankruptcies,purpose,revol_util,tax_liens,term,title,total_acc,verification_status,zip_code
index,,,,,,,,,,,,,,,,,,,,,
0,0.0,UT,80000.0,0.0,0.0,0.0,30.49,Oct-1996,4 years,689.0,...,1.0,1.0,credit_card,56.1%,0.0,36 months,Credit card refinancing,26.0,Source Verified,847xx
1,0.0,CA,82000.0,0.0,0.0,0.0,7.00,Mar-1993,10+ years,689.0,...,1.0,1.0,credit_card,88.3%,0.0,36 months,Credit card refinancing,11.0,Not Verified,900xx
2,0.0,NV,46080.0,0.0,0.0,1.0,17.32,Nov-2000,3 years,674.0,...,0.0,0.0,credit_card,18.1%,0.0,36 months,Credit card refinancing,31.0,Source Verified,895xx
3,0.0,AZ,30000.0,0.0,0.0,0.0,7.80,Jan-2010,< 1 year,694.0,...,0.0,0.0,car,33.4%,0.0,36 months,Car financing,24.0,Source Verified,853xx
4,0.0,OH,70000.0,0.0,0.0,0.0,13.36,Jan-2002,< 1 year,674.0,...,0.0,0.0,other,88.6%,0.0,60 months,Other,16.0,Not Verified,453xx
5,0.0,TX,69454.0,0.0,0.0,0.0,7.60,Apr-1993,10+ years,704.0,...,0.0,0.0,debt_consolidation,45.3%,0.0,60 months,Debt Consolitation,21.0,Verified,767xx
6,0.0,WA,78000.0,0.0,0.0,0.0,10.82,Apr-2001,5 years,664.0,...,0.0,0.0,debt_consolidation,24.1%,0.0,60 months,Debt consolidation,29.0,Source Verified,983xx
7,0.0,FL,55000.0,0.0,0.0,1.0,17.11,Sep-1995,3 years,699.0,...,1.0,1.0,debt_consolidation,55.2%,0.0,36 months,Debt consolidation,11.0,Source Verified,322xx
8,0.0,MA,60000.0,0.0,0.0,0.0,7.79,Nov-1991,10+ years,679.0,...,0.0,0.0,debt_consolidation,85.5%,0.0,36 months,Debt consolidation,7.0,Source Verified,019xx


In [14]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1199861 entries, 0 to 1199860
Data columns (total 44 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   acc_now_delinq              1199861 non-null  float64
 1   addr_state                  1199861 non-null  object 
 2   annual_inc                  1199861 non-null  float64
 3   chargeoff_within_12_mths    1199824 non-null  float64
 4   collections_12_mths_ex_med  1199824 non-null  float64
 5   delinq_2yrs                 1199861 non-null  float64
 6   dti                         1199861 non-null  float64
 7   earliest_cr_line            1199861 non-null  object 
 8   emp_length                  1128114 non-null  object 
 9   fico_range_high             1199861 non-null  float64
 10  fico_range_low              1199861 non-null  float64
 11  funded_amnt                 1199861 non-null  float64
 12  home_ownership              1199861 non-null  object 
 13  in

**Описание данных:**
- `acc_now_delinq` - Кол-во счетов, по которым заемщик сейчас просрочен
- `addr_state` - Штат заемщика, указанны в кредитной заявке
- `annual_inc` - Годовой доход заёмщика
- `chargeoff_within_12_mths` - Кол-во списаний за 12 месяцев
- `collections_12_mths_ex_med` - Кол-во списаний за 12 месяцев (без учета медицинских сборов)
- `delinq_2yrs` - Кол-во просроченных задолженностей (более чем на 30 дней) за последние 2 года
- `dti` - Соотношение общей суммы ежемесячных платежей заемщика к общей сумме долговых обязательств, за исключением ипотечного кредита и запрошенного кредита, на ежемесячный доход заемщика, о котором сообщается самостоятельно
- `earliest_cr_line` - Месяц, в котором заемщик открыл самую раннюю из заявленных кредитных линий
- `emp_length` - Стаж работы в годах. Возможные значения находятся в диапазоне от 0 до 10, где 0 означает менее одного года, а 10 означает десять или более лет
- `fico_range_high` - Верхняя граничница кредитного рейтинга **FICO** заемщика при выдаче кредита.
- `fico_range_low` - Нижняя граничница кредитного рейтинга **FICO** заемщика при выдаче кредита
- `funded_amnt` - Общая сумма обязательств по этому кредиту на данный момент времени
- `home_ownership` - Статус собственности на жилье, предоставленный заемщиком при регистрации или полученный из кредитного отчета. Возможные значения: RENT, OWN, MORTGAGE, OTHER.
- `inq_last_12m` - Кол-во кредитных запросов за последние 12 месяцев
- `installment` - Ежемесячный платеж, причитающийся заемщику, если кредит выдан.
- `int_rate` - Процентная ставка по кредиту
- `issue_d` - Месяц, в котором был выдан кредит
- `loan_amnt` - Перечисленная сумма кредита, заявленного заемщиком. Если в какой-то момент кредитный отдел уменьшит сумму кредита, то это отразится на этом значении.
- `mort_acc` - Кол-во ипотечных счетов
- `mths_since_last_delinq` - Кол-во месяцев с момента последней просрочки заемщиком.
- `mths_since_recent_bc_dlq` - Кол-во месяцев с момента последнего просроченного платежа по банковской карте
- `mths_since_recent_inq` - Кол-во месяцев с момента последнего запроса.
- `num_accts_ever_120_pd` - Кол-во счетов, просроченных на 120 и более дней
- `num_actv_bc_tl` - Кол-во активных на данный момент банковских карт
- `num_rev_accts` - Кол-во оборотных счетов
- `num_sats` - Кол-во погашеных кред. счетов
- `num_tl_120dpd_2m` - Кол-во счетов, в настоящее время просроченных на 120 дней (обновлено за последние 2 месяца)
- `num_tl_30dpd` - Кол-во счетов, в настоящее время просроченных на 30 дней (обновлено за последние 2 месяца)
- `num_tl_90g_dpd_24m` - Кол-во счетов, просроченных на 90 и более дней за последние 24 месяца
- `num_tl_op_past_12m` - Кол-во счетов, открытых за последние 12 месяцев
- `open_acc` - Кол-во открытых кредитных линий в кредитном досье заемщика.
- `open_il_24m` - Кол-во счетов в рассрочку, открытых за последние 24 месяца
- `open_rv_24m` - Кол-во возобновляемых сделок, открытых за последние 24 месяца
- `percent_bc_gt_75` - Процент всех счетов банковских карт > 75% от лимита.
- `pub_rec` - Кол-во уничижительный публичных записей
- `pub_rec_bankruptcies` - Кол-во публичных банкротств
- `purpose` - Категория, предоставляемая заемщиком для заявки на кредит
- `revol_util` - Коэффициент использования возобновляемой линии или сумма кредита, которую заемщик использует по отношению ко всему доступному возобновляемому кредиту.
- `tax_liens` - Кол-во налоговых залогов
- `term` - Кол-во платежей по кредиту. Значения указаны в месяцах и могут быть 36 или 60.
- `title` - Название кредита, предоставленное заемщиком
- `total_acc` - Общее количество кредитных линий в настоящее время в кредитном деле заемщика
- `verification_status` - Указывает: был ли доход подтвержден LC, не подтвержден.
- `zip_code` - Первые 3 цифры почтового индекса, указанные заемщиком в кредитной заявке.

In [4]:
X_train.columns

Index(['index', 'acc_now_delinq', 'addr_state', 'annual_inc',
       'chargeoff_within_12_mths', 'collections_12_mths_ex_med', 'delinq_2yrs',
       'dti', 'earliest_cr_line', 'emp_length', 'fico_range_high',
       'fico_range_low', 'funded_amnt', 'home_ownership', 'inq_last_12m',
       'installment', 'int_rate', 'issue_d', 'loan_amnt', 'mort_acc',
       'mths_since_last_delinq', 'mths_since_recent_bc_dlq',
       'mths_since_recent_inq', 'num_accts_ever_120_pd', 'num_actv_bc_tl',
       'num_rev_accts', 'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd',
       'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'open_acc', 'open_il_24m',
       'open_rv_24m', 'percent_bc_gt_75', 'pub_rec', 'pub_rec_bankruptcies',
       'purpose', 'revol_util', 'tax_liens', 'term', 'title', 'total_acc',
       'verification_status', 'zip_code'],
      dtype='object')

## Предобработка данных

Типы данных, пропуски, дубликаты, аномалии/выбросы.

## Анализ данных (EDA)

## Модели предсказаний

## Вывод